# Wprowadzenie do uczenia maszynowego w języku Julia

## Dlaczego Julia

W trakcie kursu pracować będziemy w języku [Julia](https://julialang.org/). Dlaczego?

- Język skryptowy (jak Python  czy R)
- Szybkość  (jak C)
- Silny  system typów 
- Wbudowane  zrównoleglanie  obliczeń
- Łatwość  integracji  integracji (Python, R, C, …) 


Zacznijmy od omówienia prostego przykładu. Interesuje nas oszacowanie prostego modelu regresji liniowej na [danych dotyczących cen domów w Bostonie](https://www.cs.toronto.edu/~delve/data/boston/bostonDetail.html). Zacznijmy od wczytania danych:

In [1]:
using DelimitedFiles

In [2]:
#isfile("housing.data") ||
#  download("https://raw.githubusercontent.com/MikeInnes/notebooks/master/housing.data",
  #         "housing.data")
rawdata = readdlm("housing.data")'

14×505 LinearAlgebra.Adjoint{Float64,Array{Float64,2}}:
   0.02731    0.02729    0.03237  …    0.06076    0.10959    0.04741
   0.0        0.0        0.0           0.0        0.0        0.0    
   7.07       7.07       2.18         11.93      11.93      11.93   
   0.0        0.0        0.0           0.0        0.0        0.0    
   0.469      0.469      0.458         0.573      0.573      0.573  
   6.421      7.185      6.998    …    6.976      6.794      6.03   
  78.9       61.1       45.8          91.0       89.3       80.8    
   4.9671     4.9671     6.0622        2.1675     2.3889     2.505  
   2.0        2.0        3.0           1.0        1.0        1.0    
 242.0      242.0      222.0         273.0      273.0      273.0    
  17.8       17.8       18.7      …   21.0       21.0       21.0    
 396.9      392.83     394.63        396.9      393.45     396.9    
   9.14       4.03       2.94          5.64       6.48       7.88   
  21.6       34.7       33.4          23.9     

In [3]:
X = rawdata[1:13,:];
y = rawdata[14:14,:];

Obejrzyjmy wczytane dane, wykorzystamy do tego bibliotekę [<tt>DataFrames</tt>](https://juliadata.github.io/DataFrames.jl/stable/man/getting_started.html):

In [4]:
using DataFrames

In [5]:
df = DataFrames.DataFrame(vcat(X,y)')
names!(df,
    [:CRIM #- per capita crime rate by town
    :ZN #- proportion of residential land zoned for lots over 25,000 sq.ft.
    :INDUS #- proportion of non-retail business acres per town.
    :CHAS #- Charles River dummy variable (1 if tract bounds river; 0 otherwise)
    :NOX #- nitric oxides concentration (parts per 10 million)
    :RM #- average number of rooms per dwelling
    :AGE #- proportion of owner-occupied units built prior to 1940
    :DIS #- weighted distances to five Boston employment centres
    :RAD #- index of accessibility to radial highways
    :TAX #- full-value property-tax rate per $10,000
    :PTRATIO #- pupil-teacher ratio by town
    :B #- 1000(Bk - 0.63)^2 where Bk is the proportion of blacks by town
    :LSTAT #- % lower status of the population
    :MEDV]) #- Median value of owner-occupied homes in $1000's)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0
5,0.02985,0.0,2.18,0.0,0.458,6.43,58.7,6.0622,3.0
6,0.08829,12.5,7.87,0.0,0.524,6.012,66.6,5.5605,5.0
7,0.14455,12.5,7.87,0.0,0.524,6.172,96.1,5.9505,5.0
8,0.21124,12.5,7.87,0.0,0.524,5.631,100.0,6.0821,5.0
9,0.17004,12.5,7.87,0.0,0.524,6.004,85.9,6.5921,5.0


In [6]:
describe(df)

,variable,mean,min,median,max,nunique,nmissing,eltype
,Symbol,Float64,Float64,Float64,Float64,Nothing,Nothing,DataType
1,CRIM,3.62067,0.00906,0.25915,88.9762,,,Float64
2,ZN,11.3505,0.0,0.0,100.0,,,Float64
3,INDUS,11.1543,0.46,9.69,27.74,,,Float64
4,CHAS,0.0693069,0.0,0.0,1.0,,,Float64
5,NOX,0.554728,0.385,0.538,0.871,,,Float64
6,RM,6.28406,3.561,6.208,8.78,,,Float64
7,AGE,68.5816,2.9,77.7,100.0,,,Float64
8,DIS,3.79446,1.1296,3.1992,12.1265,,,Float64
9,RAD,9.56634,1.0,5.0,24.0,,,Float64


In [7]:
first(df, 5)

,CRIM,ZN,INDUS,CHAS,NOX,RM,AGE,DIS,RAD
,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64,Float64
1,0.02731,0.0,7.07,0.0,0.469,6.421,78.9,4.9671,2.0
2,0.02729,0.0,7.07,0.0,0.469,7.185,61.1,4.9671,2.0
3,0.03237,0.0,2.18,0.0,0.458,6.998,45.8,6.0622,3.0
4,0.06905,0.0,2.18,0.0,0.458,7.147,54.2,6.0622,3.0
5,0.02985,0.0,2.18,0.0,0.458,6.43,58.7,6.0622,3.0


Zdefiniujmy funkcję estymującą parametry regresji jako iloczyn macierzy:
$\hat{\beta} = (X^TX)^{-1}X^Ty $

In [8]:
function least_squares(X::Array{Float64},
    y::Array{Float64} , intercept::Bool = true)
    intercept ? (X = hcat(X, ones(size(X)[1]))) : X = X
    return (X'*X)^-1*X'*y
end

least_squares (generic function with 2 methods)

In [36]:
@time θ = least_squares(Matrix(X'),Array(y'))

  0.000189 seconds (22 allocations: 181.609 KiB)


14×1 Array{Float64,2}:
 -0.9247068981906081 
  1.0766287659838476 
  0.09782618662283815
  0.6790682306887288 
 -2.0453293772847605 
  2.668136302274164  
  0.03032232046769412
 -3.1177508810393286 
  2.6256244487812683 
 -2.0325705172693374 
 -2.073583342517735  
  0.8502139446905561 
 -3.7670350726870168 
 22.529900990099005  

In [10]:
using BenchmarkTools

┌ Info: Recompiling stale cache file C:\Users\p\.julia\compiled\v1.0\BenchmarkTools\ZXPQo.ji for BenchmarkTools [6e4b80f9-dd63-53aa-95a3-0cdb28fa8baf]
└ @ Base loading.jl:1184


In [11]:
@benchmark θ = least_squares(Matrix(X'),Matrix(y'))

BenchmarkTools.Trial: 
  memory estimate:  181.45 KiB
  allocs estimate:  18
  --------------
  minimum time:     63.100 μs (0.00% GC)
  median time:      125.499 μs (0.00% GC)
  mean time:        166.216 μs (13.89% GC)
  maximum time:     69.541 ms (99.75% GC)
  --------------
  samples:          10000
  evals/sample:     1

Porównajmy otrzymany wynik z wynikiem otrzymanym za pomocą biblioteki <tt>GLM</tt>:

In [12]:
using GLM

┌ Info: Recompiling stale cache file C:\Users\p\.julia\compiled\v1.0\GLM\6OREG.ji for GLM [38e38edf-8417-5370-95a0-9cbb8c7f171a]
└ @ Base loading.jl:1184


In [13]:
ols = lm(@formula(MEDV ~ CRIM +ZN + INDUS + CHAS + NOX + RM +
    AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT  + 1), df)

┌ Warning: `T` is deprecated, use `nonmissingtype` instead.
│   caller = (::getfield(StatsModels, Symbol("##40#41")){Array{Int64,1}})(::Array{Float64,1}) at none:0
└ @ StatsModels .\none:0


StatsModels.TableRegressionModel{LinearModel{GLM.LmResp{Array{Float64,1}},GLM.DensePredChol{Float64,LinearAlgebra.Cholesky{Float64,Array{Float64,2}}}},Array{Float64,2}}

MEDV ~ 1 + CRIM + ZN + INDUS + CHAS + NOX + RM + AGE + DIS + RAD + TAX + PTRATIO + B + LSTAT

Coefficients:
───────────────────────────────────────────────────────────────────────────────────────
                 Estimate  Std. Error     t value  Pr(>|t|)     Lower 95%     Upper 95%
───────────────────────────────────────────────────────────────────────────────────────
(Intercept)   36.6349      5.10204       7.18045     <1e-11   26.6104       46.6595    
CRIM          -0.107417    0.0328473    -3.2702      0.0012   -0.171956     -0.0428785 
ZN             0.0461207   0.0137207     3.3614      0.0008    0.0191622     0.0730792 
INDUS          0.014269    0.0616534     0.231438    0.8171   -0.106868      0.135406  
CHAS           2.67111     0.861115      3.10192     0.0020    0.979182      4.36303   
NOX          -17.6

Możemy też zdefiniować problem oszacowania parametrów tej regresji  jako zadanie optymalizacyjne:



Zacznijmy od znormalizowania zmiennych:

In [19]:
using StatsBase: mean, std

In [20]:
X = (X .- mean(X, dims = 2)) ./ std(X, dims = 2);

i wyznaczenia początkowych współczynników oraz zdefiniowania funkcji straty $J(\theta)$:

In [21]:
W = rand(1,13);
b = rand();

In [22]:
#predict(W, b, x) = W * x .+ b
Predict(W, b, x) = W * x .+ b
MSE(ŷ, y) = sum((ŷ .- y).^2)/size(y, 2)

MSE (generic function with 1 method)

In [23]:
MSE(Predict(W,b,X), y)

590.9280977643542

Do rozwiązania zadanego problemu wykorzystamy [metodę gradientu prostego](https://pl.wikipedia.org/wiki/Metoda_gradientu_prostego). Zdefiniujmy funkcję, która wyznacza gradient:

In [24]:
function simple_∇(W::Array{Float64}, b::Float64, x::Array{Float64}, 
        y::Array{Float64}; Δ::Float64 = 0.000000001)
    ∇ = Float64[] 
    for i = 1:length(W)
        W′ = W .+ (LinearIndices(W) .== i) * Δ  
        df = (MSE(Predict(W′,b,x),y) - MSE(Predict(W, b ,x),y)) /  Δ
        push!(∇,df)
    end
    ∇b = (MSE(Predict(W,b + Δ,x),y) - MSE(Predict(W,b ,x),y)) /  Δ
    return ∇, ∇b
end

simple_∇ (generic function with 1 method)

In [25]:
simple_∇(W,b,X,y)

([10.1086, -7.20888, 11.9295, -2.81773, 10.4103, -14.2139, 9.40372, -6.16933, 10.5987, 12.444, 12.2351, -7.46275, 16.626], -43.672571337083355)

Zdefiniujmy funkcję rozwiązującą zadanie:

In [26]:
function solve(W::Array{Float64}, b::Float64, x::Array{Float64}, y::Array{Float64};
                η::Float64 = 0.001, ϵ::Float64 = 10^-10, maxit::Int64 = 50_000)
    iter = 1
    θ = W
    β = b
    ∇,∇b = simple_∇(W,b,x,y)
    while true
        θ₀, β₀ = θ, β
        θ -= η * ∇'
        β  -= η * ∇b
        ∇,∇b = simple_∇(θ,β,x,y)
        stop = maximum(abs.(hcat(θ, β) .- hcat(θ₀, β₀)))
        stop < ϵ && break
        iter += 1
        iter > maxit && break
    end
    println(iter)
    return θ, β
end

solve (generic function with 1 method)

In [27]:
θ, β = solve(W,b,X,y)

50001


([-0.924399 1.07609 … 0.850184 -3.76688], 22.52990111449347)

In [28]:
MSE(Predict(θ,β,X), y)

21.865582762288962

Problematyczną kwestią jest sposób numerycznego liczenia pochodnych. Klasyczny sposób ich definiowania jest nieefektywny numerycznie. Koniecznie jest wykorzystanie [alternatywnych metod liczenia](http://www.juliadiff.org/). Niestety są one zazwyczaj trudne do samodzielnej implementacji i warto wtedy skorzystać z jednego z zaimplementowanych gdzie indziej rozwiązań.

Rozwiążmy zadanie jeszcze raz, tym razem wykorzystując do tego bibliotekę [<tt>Flux</tt>](https://fluxml.ai/):

In [29]:
using Flux.Tracker, Statistics, DelimitedFiles
using Flux.Tracker: Params, gradient, update!
using Flux: gpu

┌ Info: Recompiling stale cache file C:\Users\p\.julia\compiled\v1.0\Flux\QdkVy.ji for Flux [587475ba-b771-5e3f-ad9e-33799f191a9c]
└ @ Base loading.jl:1184


In [30]:
W = param(randn(1,13)/10) 
b = param([0.]) 

Tracked 1-element Array{Float64,1}:
 0.0

In [31]:
loss(x, y) = MSE(Predict(W,b,x), y)

loss (generic function with 1 method)

In [32]:
η = 0.001
ϵ =  10^-10
θ = Params([W, b])

Params([[0.0338516 -0.0965621 … 0.0621878 0.0326175] (tracked), [0.0] (tracked)])

In [33]:
while true
    g = gradient(() -> loss(X, y), θ)
    θ₀ = hcat(θ...)
    for x in θ
        update!(x, -g[x]*η)
    end
    @show loss(X,y)
    stop = maximum(abs.(θ₀ .- hcat(θ...)))
    stop < ϵ && break
end

loss(X, y) = 587.937931032034 (tracked)
loss(X, y) = 585.0957713283731 (tracked)
loss(X, y) = 582.2789045886107 (tracked)
loss(X, y) = 579.4868934120983 (tracked)
loss(X, y) = 576.7193103059491 (tracked)
loss(X, y) = 573.9757374472157 (tracked)
loss(X, y) = 571.2557664508302 (tracked)
loss(X, y) = 568.5589981431776 (tracked)
loss(X, y) = 565.885042341155 (tracked)
loss(X, y) = 563.2335176365912 (tracked)
loss(X, y) = 560.6040511858912 (tracked)
loss(X, y) = 557.9962785047842 (tracked)
loss(X, y) = 555.4098432680445 (tracked)
loss(X, y) = 552.8443971140691 (tracked)
loss(X, y) = 550.299599454194 (tracked)
loss(X, y) = 547.7751172866308 (tracked)
loss(X, y) = 545.2706250149153 (tracked)
loss(X, y) = 542.7858042707551 (tracked)
loss(X, y) = 540.3203437411718 (tracked)
loss(X, y) = 537.8739389998326 (tracked)
loss(X, y) = 535.4462923424688 (tracked)
loss(X, y) = 533.0371126262828 (tracked)
loss(X, y) = 530.6461151132474 (tracked)
loss(X, y) = 528.2730213172007 (tracked)
loss(X, y) = 525.91

loss(X, y) = 243.00088133930305 (tracked)
loss(X, y) = 242.11850170394808 (tracked)
loss(X, y) = 241.23973906573704 (tracked)
loss(X, y) = 240.36457733157914 (tracked)
loss(X, y) = 239.49300050852486 (tracked)
loss(X, y) = 238.62499270251706 (tracked)
loss(X, y) = 237.7605381171688 (tracked)
loss(X, y) = 236.89962105256507 (tracked)
loss(X, y) = 236.04222590408912 (tracked)
loss(X, y) = 235.18833716127267 (tracked)
loss(X, y) = 234.33793940666928 (tracked)
loss(X, y) = 233.49101731474977 (tracked)
loss(X, y) = 232.64755565082095 (tracked)
loss(X, y) = 231.8075392699642 (tracked)
loss(X, y) = 230.970953115997 (tracked)
loss(X, y) = 230.13778222045335 (tracked)
loss(X, y) = 229.30801170158563 (tracked)
loss(X, y) = 228.48162676338518 (tracked)
loss(X, y) = 227.65861269462252 (tracked)
loss(X, y) = 226.83895486790604 (tracked)
loss(X, y) = 226.02263873875907 (tracked)
loss(X, y) = 225.2096498447151 (tracked)
loss(X, y) = 224.39997380442972 (tracked)
loss(X, y) = 223.59359631681045 (tracke

loss(X, y) = 119.98055929289939 (tracked)
loss(X, y) = 119.59563551344871 (tracked)
loss(X, y) = 119.21225660408939 (tracked)
loss(X, y) = 118.83041631631639 (tracked)
loss(X, y) = 118.45010842720677 (tracked)
loss(X, y) = 118.07132673930998 (tracked)
loss(X, y) = 117.69406508053868 (tracked)
loss(X, y) = 117.31831730405993 (tracked)
loss(X, y) = 116.94407728818727 (tracked)
loss(X, y) = 116.57133893627301 (tracked)
loss(X, y) = 116.20009617660146 (tracked)
loss(X, y) = 115.83034296228256 (tracked)
loss(X, y) = 115.46207327114573 (tracked)
loss(X, y) = 115.09528110563491 (tracked)
loss(X, y) = 114.72996049270361 (tracked)
loss(X, y) = 114.36610548371081 (tracked)
loss(X, y) = 114.0037101543171 (tracked)
loss(X, y) = 113.6427686043816 (tracked)
loss(X, y) = 113.2832749578593 (tracked)
loss(X, y) = 112.92522336269906 (tracked)
loss(X, y) = 112.5686079907417 (tracked)
loss(X, y) = 112.21342303761914 (tracked)
loss(X, y) = 111.85966272265365 (tracked)
loss(X, y) = 111.50732128875785 (track

loss(X, y) = 65.58523660257299 (tracked)
loss(X, y) = 65.41803223083195 (tracked)
loss(X, y) = 65.25149454322556 (tracked)
loss(X, y) = 65.08562085770303 (tracked)
loss(X, y) = 64.92040850305348 (tracked)
loss(X, y) = 64.75585481886215 (tracked)
loss(X, y) = 64.5919571554665 (tracked)
loss(X, y) = 64.42871287391237 (tracked)
loss(X, y) = 64.26611934591085 (tracked)
loss(X, y) = 64.1041739537944 (tracked)
loss(X, y) = 63.9428740904743 (tracked)
loss(X, y) = 63.78221715939724 (tracked)
loss(X, y) = 63.62220057450274 (tracked)
loss(X, y) = 63.46282176018055 (tracked)
loss(X, y) = 63.30407815122806 (tracked)
loss(X, y) = 63.14596719280815 (tracked)
loss(X, y) = 62.988486340407086 (tracked)
loss(X, y) = 62.83163305979261 (tracked)
loss(X, y) = 62.675404826972084 (tracked)
loss(X, y) = 62.51979912815107 (tracked)
loss(X, y) = 62.36481345969179 (tracked)
loss(X, y) = 62.21044532807194 (tracked)
loss(X, y) = 62.056692249843664 (tracked)
loss(X, y) = 61.90355175159255 (tracked)
loss(X, y) = 61.

loss(X, y) = 42.41616035303786 (tracked)
loss(X, y) = 42.34100909545201 (tracked)
loss(X, y) = 42.26615479556843 (tracked)
loss(X, y) = 42.19159626327537 (tracked)
loss(X, y) = 42.117332313254316 (tracked)
loss(X, y) = 42.0433617649604 (tracked)
loss(X, y) = 41.969683442603234 (tracked)
loss(X, y) = 41.89629617512761 (tracked)
loss(X, y) = 41.82319879619451 (tracked)
loss(X, y) = 41.75039014416183 (tracked)
loss(X, y) = 41.67786906206552 (tracked)
loss(X, y) = 41.605634397600646 (tracked)
loss(X, y) = 41.53368500310258 (tracked)
loss(X, y) = 41.46201973552814 (tracked)
loss(X, y) = 41.39063745643703 (tracked)
loss(X, y) = 41.319537031973205 (tracked)
loss(X, y) = 41.24871733284626 (tracked)
loss(X, y) = 41.178177234313104 (tracked)
loss(X, y) = 41.107915616159424 (tracked)
loss(X, y) = 41.03793136268154 (tracked)
loss(X, y) = 40.96822336266803 (tracked)
loss(X, y) = 40.898790509381634 (tracked)
loss(X, y) = 40.829631700541285 (tracked)
loss(X, y) = 40.76074583830387 (tracked)
loss(X, y

...

loss(X, y) = 21.899208779057755 (tracked)
loss(X, y) = 21.899200005039596 (tracked)
loss(X, y) = 21.89919123342716 (tracked)
loss(X, y) = 21.89918246421971 (tracked)
loss(X, y) = 21.89917369741651 (tracked)
loss(X, y) = 21.899164933016824 (tracked)
loss(X, y) = 21.89915617101991 (tracked)
loss(X, y) = 21.899147411425044 (tracked)
loss(X, y) = 21.89913865423147 (tracked)
loss(X, y) = 21.899129899438478 (tracked)
loss(X, y) = 21.899121147045303 (tracked)
loss(X, y) = 21.89911239705122 (tracked)
loss(X, y) = 21.8991036494555 (tracked)
loss(X, y) = 21.899094904257417 (tracked)
loss(X, y) = 21.89908616145621 (tracked)
loss(X, y) = 21.899077421051164 (tracked)
loss(X, y) = 21.899068683041545 (tracked)
loss(X, y) = 21.899059947426604 (tracked)
loss(X, y) = 21.899051214205624 (tracked)
loss(X, y) = 21.899042483377862 (tracked)
loss(X, y) = 21.899033754942586 (tracked)
loss(X, y) = 21.899025028899075 (tracked)
loss(X, y) = 21.899016305246576 (tracked)
loss(X, y) = 21.899007583984368 (tracked)
l

loss(X, y) = 21.896981937922938 (tracked)
loss(X, y) = 21.896973771996006 (tracked)
loss(X, y) = 21.896965608289918 (tracked)
loss(X, y) = 21.89695744680399 (tracked)
loss(X, y) = 21.89694928753757 (tracked)
loss(X, y) = 21.896941130489967 (tracked)
loss(X, y) = 21.89693297566053 (tracked)
loss(X, y) = 21.896924823048575 (tracked)
loss(X, y) = 21.89691667265344 (tracked)
loss(X, y) = 21.896908524474473 (tracked)
loss(X, y) = 21.896900378510978 (tracked)
loss(X, y) = 21.896892234762312 (tracked)
loss(X, y) = 21.896884093227793 (tracked)
loss(X, y) = 21.896875953906754 (tracked)
loss(X, y) = 21.896867816798537 (tracked)
loss(X, y) = 21.89685968190248 (tracked)
loss(X, y) = 21.8968515492179 (tracked)
loss(X, y) = 21.896843418744133 (tracked)
loss(X, y) = 21.89683529048052 (tracked)
loss(X, y) = 21.896827164426405 (tracked)
loss(X, y) = 21.896819040581107 (tracked)
loss(X, y) = 21.896810918943974 (tracked)
loss(X, y) = 21.89680279951432 (tracked)
loss(X, y) = 21.896794682291496 (tracked)
l

loss(X, y) = 21.895384348669904 (tracked)
loss(X, y) = 21.89537661599347 (tracked)
loss(X, y) = 21.895368885408082 (tracked)
loss(X, y) = 21.895361156913108 (tracked)
loss(X, y) = 21.895353430507935 (tracked)
loss(X, y) = 21.89534570619194 (tracked)
loss(X, y) = 21.895337983964478 (tracked)
loss(X, y) = 21.895330263824956 (tracked)
loss(X, y) = 21.895322545772732 (tracked)
loss(X, y) = 21.895314829807194 (tracked)
loss(X, y) = 21.895307115927725 (tracked)
loss(X, y) = 21.8952994041337 (tracked)
loss(X, y) = 21.8952916944245 (tracked)
loss(X, y) = 21.895283986799498 (tracked)
loss(X, y) = 21.895276281258088 (tracked)
loss(X, y) = 21.89526857779963 (tracked)
loss(X, y) = 21.89526087642352 (tracked)
loss(X, y) = 21.895253177129128 (tracked)
loss(X, y) = 21.895245479915854 (tracked)
loss(X, y) = 21.895237784783053 (tracked)
loss(X, y) = 21.89523009173012 (tracked)
loss(X, y) = 21.895222400756438 (tracked)
loss(X, y) = 21.89521471186139 (tracked)
loss(X, y) = 21.895207025044346 (tracked)
lo

Excessive output truncated after 524296 bytes.


loss(X, y) = 21.89395934404208 (tracked)
loss(X, y) = 21.893951995752595 (tracked)
loss(X, y) = 21.893944649440318 (tracked)
loss(X, y) = 21.893937305104682 (tracked)
loss(X, y) = 21.893929962745087 (tracked)
loss(X, y) = 21.893922622360968 (tracked)
loss(X, y) = 21.893915283951724 (tracked)
loss(X, y) = 21.893907947516798 (tracked)
loss(X, y) = 21.893900613055596 (tracked)
loss(X, y) = 21.893893280567532 (tracked)
loss(X, y) = 21.893885950052027 (tracked)
loss(X, y) = 21.893878621508502 (tracked)
loss(X, y) = 21.893871294936393 (tracked)

In [35]:
θ

Params([[-0.924707 1.07663 … 0.850214 -3.76704] (tracked), [22.5299] (tracked)])

In [34]:
MSE(Predict(θ...,X), y)

21.865579933351484 (tracked)

## Co dalej?

W ramach dzisiejszego prostego przykładu pokazaliśmy sobie to w jaki sposób można budować modele predykcyjne wykorzystując do tego najbardziej podstawowe narzędzia wbudowane w Julię. W celu poszerzenia swojej wiedzy 
Aby być jednak w stanie budować bardziej złożone modele potrzebować będziemy odpowiednich bibliotek ułatwiających nam pracę, między innymi: 

###  1. DataFrames.jl

Biblioteka [<tt>DataFrames</tt>](https://juliadata.github.io/DataFrames.jl/stable/man/getting_started.html) jest narzędziem pozwalającym na efektywną i wygodną pracę ze zbiorami danych. Jest implementacja znanych z <tt>R</tt> ramek danych, oferując wszystkie znane z <tt>R</tt> narzędzia, zaimplementowane w wyraźnie efektywniejszy obliczeniowo sposób. Warto zapoznać się ze szczegółowym [wprowadzeniem do <tt>DataFrames</tt> ](https://github.com/bkamins/Julia-DataFrames-Tutorial).

### 2. Plots.jl

[Plots]() to podstawowa bibliotka do tworzenia wykresów w Julii. Jedną z jej głównych zalet jest to, że pozwala na wykorzystanie wielu [backendów](http://docs.juliaplots.org/latest/backends/). Warto zapoznać się z [dokumenacją](http://docs.juliaplots.org/latest/) tej biblioteki.

### 3. Flux.jl

W celu zapoznania się z działaniem biblioteki <tt>Flux</tt> warto zajrzeć do jej [dokumentacji](https://fluxml.ai/Flux.jl/stable/) i zapoznać się z załączonym do dzisiejszych notatek skryptem.

## Literatura 

Poza wymienioną powyżej literaturą wprowadzającą do specyficznych zagadnień potrzebnych podczas tego kursu istenieje wiele dodatkowych przydatnych źródeł pozwalających rozwinąć swoją wiedzę na temat programowania w Julii:
- [Julia Express](https://github.com/bkamins/The-Julia-Express)
- [wykłady Quantitative Economics Sargenta i Stachurskiego](https://lectures.quantecon.org/jl/)
- [Julia dla Data Science](http://ucidatascienceinitiative.github.io/IntroToJulia/)
- [Think Julia](https://benlauwens.github.io/ThinkJulia.jl/latest/book.html)
- [materiały dostępne na stronie języka](https://julialang.org/learning/)

## Dodatkowe prace domowe

1. Przepisz zaprezentowaną powyżej implementacje klasycznej metody najmniejszych kwadratów na języki <tt>Python</tt> i <tt>R</tt>. Porównaj czasy rozwiązania problemu.

2. Zaprezentowana na zajęciach implementacja estymacji parametrów modelu za pomocą klasycznej metody najmniejszych kwadratów nie jest efektywnym numerycznie rozwiązaniem problemu. Postaraj się zmodyfikować je w taki sposób aby było szybsze niż rozwiązanie prezentowane na zajęciach. Porównaj oba sposoby zaimplementowania algorytmu.
